<a href="https://colab.research.google.com/github/TitaniumCoderr/Capstone/blob/master/model_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D,GlobalAvgPool2D, Input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import callbacks, optimizers
import numpy as np
from google.colab import drive
import tensorflow_datasets as tfds #imported to get the plant_village dataset
import matplotlib.pyplot as plt
import random 
 
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split #split the data
from tensorflow.keras.models import Sequential

In [ ]:
#mountoing the drive to access the dataset
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/



/content/drive/MyDrive


In [ ]:
#listing the classes in the dataset
for i in os.listdir("Plant_dataset"):
  print(i, len(os.listdir("Plant_dataset/"+i)))


In [ ]:
gen_object = ImageDataGenerator

In [ ]:
#pre-proccessing 
#generates and returns an image data generator object
def img_data(dir_path,target_size,batch,class_lst,preprocesss):
  if preprocesss:
    gen_object= ImageDataGenerator(preprocessing_function=preprocesss)
  else:
    gen_object=ImageDataGenerator()

  return (gen_object.flow_from_directory(dir_path,
                                         target_size=target_size,
                                         batch_size=batch,
                                         class_mode='sparse',  
                                         classes=class_lst,
                                         shuffle=True))


In [ ]:
#pre-proccessing 

train_data_gen = img_data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen = img_data("test",(224,224),500,os.listdir("test"),preprocess_input)

In [ ]:
#pre-trained base model
my_model= tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax')

In [ ]:
my_model.trainable=False

In [ ]:
#building the model
model= tf.keras.models.Sequential()
model.add(my_model) # Adding the MobileNetV2 model as a base layer
model.add(GlobalAvgPool2D()) # Global average pooling layer
model.add(Dense(1024, activation='relu')) # Fully connected layer with 1024 units and ReLU activation
model.add(Dense(25, activation='softmax')) # Output layer with 25 units and softmax activation

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
elst= callbacks.EarlyStopping(monitor='val_loss',patience=5, mode='min') 
save_ck= callbacks.ModelCheckpoint(".myModel.hdf5", save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
model.fit(train_data_gen, batch_size=500, validation_data=valid_data_gen,callbacks=[elst,save_ck], epochs=10)

Epoch 1/10
15/15 [==============================] - 2282s 153s/step - loss: 1.0284 - accuracy: 0.7313 - val_loss: 0.3133 - val_accuracy: 0.8970
Epoch 2/10
15/15 [==============================] - 861s 57s/step - loss: 0.1795 - accuracy: 0.9442 - val_loss: 0.1468 - val_accuracy: 0.9531
Epoch 3/10
15/15 [==============================] - 861s 58s/step - loss: 0.0890 - accuracy: 0.9733 - val_loss: 0.1266 - val_accuracy: 0.9582
Epoch 4/10
15/15 [==============================] - 871s 58s/step - loss: 0.0641 - accuracy: 0.9824 - val_loss: 0.1070 - val_accuracy: 0.9683
Epoch 5/10
15/15 [==============================] - 874s 59s/step - loss: 0.0432 - accuracy: 0.9905 - val_loss: 0.1032 - val_accuracy: 0.9691
Epoch 6/10
15/15 [==============================] - 864s 60s/step - loss: 0.0289 - accuracy: 0.9950 - val_loss: 0.0814 - val_accuracy: 0.9745
Epoch 7/10
15/15 [==============================] - 872s 58s/step - loss: 0.0191 - accuracy: 0.9980 - val_loss: 0.0743 - val_accuracy: 0.9756
Epoc